# 巡回セールスマン問題を計算する

巡回セールスマン問題とは、訪れる都市の集合と都市間の移動コスト（距離など）が与えられたとき、すべての都市を１度ずつ訪問し、かつ移動コスト（距離）が最小のルートを求めるという組み合わせ最適化問題です。 

blueqatがインストールされていない場合は、環境に併せて以下のようにインストールしてください。

In [1]:
!pip3 install blueqat

     |████████████████████████████████| 51kB 1.7MB/s 


必要なライブラリをimportし、wildqatオブジェクトをインスタンス化します。

In [0]:
import blueqat.wq as wq
import numpy as np
a = wq.Opt()


## 例題

簡単のために、例題で考えてみます。

図のような４都市A、B、C、Dを１度ずつ訪問することを考えます。都市間はすべて接続されていて、距離が設定されています。接続されていない都市間がある場合も解けますが、簡単のため全結合とします。

<img src="https://user-images.githubusercontent.com/5043340/45661003-8ba0cf00-bb36-11e8-95fc-573e77ded327.png" width="400">


## Quboマトリクスを作成

問題を解くためのQuboマトリクスを作成します。
巡回セールスマン問題を解くための一般式は次のようになります。
$H = \sum_{v=1}^n\left( 1-\sum_{j=1}^N x_{v,j} \right)^2 + \sum_{j=1}^N\left(1-\sum_{v=1}^Nx_{v,j} \right)^2 + B\sum_{(u,v)\in E}W_{u,v}\sum_{j=1}^N x_{u,j} x_{v,j+1}$      ・・・・・(1)



変数 $x_{vj}$ を、都市$v$を$j$番目に訪れるかどうかを表す変数とし、

$x_{vj} =  1$ (都市vをj番目に訪れるとき)、$0$ (都市vをj番目に訪れないとき)

とします。このため、Quboマトリクスは、都市数をNとすると${N}^2$×${N}^2$のマトリクスとなります。上記の４都市の例では、１６×１６のマトリクスです。Quboマトリクスは２次元ですので、$x_{vj}$を$q_i$に変換して考えます。すなわち、上記の４都市の例では、

$x_{11}, x_{12}, x_{13}, x_{14}$ →  $q_0, q_1, q_2, q_3$

$x_{21}, x_{22}, x_{23}, x_{24}$ →  $q_4, q_5, q_6, q_7$

$x_{31}, x_{32}, x_{33}, x_{34}$ →  $q_8, q_{9}, q_{10}, q_{11}$

$x_{41}, x_{42}, x_{43}, x_{44}$ →  $q_{12}, q_{13}, q_{14}, q_{15}$


といった感じです。（$x$の添え字の１番目は、1：A、2：B、3：C、4：Dとしています。）

この変数を利用して、すべての都市を１度ずつ訪れたときの移動距離が一番小さくなるようなコスト関数を考えます。コスト関数には３つの項目が必要です。


*   各都市を訪れるのは１回ずつである
*   j番目に訪れる都市は１つだけである（同じ時間に複数の都市を訪問していることはできないため）
*  移動距離が最小




## 条件１：各都市を訪れるのは１回ずつである

まず、１つ目の制約条件について考えます。下図は、都市AからDを1~4の何番目に訪れるか表した図です。
<img src="https://user-images.githubusercontent.com/5043340/45663268-8a749f80-bb40-11e8-8c4a-8b2ad1dd3f35.png" width="400">

１都市を訪れるのは１度だけと考えると、赤枠で囲んだ横の各行は、１つだけが１になります。
たとえば、$q_0+q_1+q_2+q_3 = 1$です。これをすべての行で考え、以下のようにすれば

${(1-q_0-q_1-q_2-q_3)^2+(1-q_4-q_5-q_6-q_7)^2+(1-q_8-q_9-q_{10}-q_{11})^2+(1-q_{12}-q_{13}-q_{14}-q_{15})^2
}$

上式のようにすれば、各都市を一度だけ訪れる場合に最小となる制約条件になります。





## 条件２： j番目に訪れる都市は１つだけである

２つ目の制約条件について考えます。
<img src="https://user-images.githubusercontent.com/5043340/45666641-1bec0d80-bb51-11e8-87f7-0d1bb522f2e8.png" width="400">

今度は、１度に訪れる都市は１つだけと考えると、赤枠で囲んだ縦の各列が、１つだけ１になります。そうすると、

${(1-q_0-q_4-q_8-q_{12})^2+(1-q_1-q_5-q_9-q_{13})^2+(1-q_2-q_6-q_{10}-q_{14})^2+(1-q_{3}-q_{7}-q_{11}-q_{15})^2
}$

上式のようにすれば、一度に訪れる都市が１つだけの場合に最小となる制約条件になります。



条件１と条件２を足し合わせて整理すると、以下のようになります。

${2q_0q_1 + 2q_0q_{12} + 2q_0q_2 + 2q_0q_3 + 2q_0q_4 + 2q_0q_8 - 2q_0}$ 

${+ 2q_1q_{13} + 2q_1q_2 + 2q_1q_3 + 2q_1q_5 + 2q_1q_9 - 2q_1}$ 

${ + 2q_{10}q_{11} + 2q_{10}q_{14} + 2q_{10}q_2 + 2q_{10}q_6 + 2q_{10}q_8 + 2q_{10}q_9 - 2q_{10} }$ 

${+ 2q_{11}q_{15} + 2q_{11}q_3 + 2q_{11}q_7 + 2q_{11}q_8 + 2q_{11}q_9 - 2q_{11}}$ 

${+ 2q_{12}q_{13} + 2q_{12}q_{14} + 2q_{12}q_{15} + 2q_{12}q_4 + 2q_{12}q_8 - 2q_{12} }$ 

${+ 2q_{13}q_{14}+ 2q_{13}q_{15} + 2q_{13}q_5 + 2q_{13}q_9 - 2q_{13} }$ 

${+ 2q_{14}q_{15} + 2q_{14}q_2 + 2q_{14}q_6 - 2q_{14}}$ 

${+ 2q_{15}q_3 + 2q_{15}q_7 - 2q_{15}}$ 

${+ 2q_2q_3 + 2q_2q_6 - 2q_2 + 2q_3q_7 - 2q_3 }$ 

${+ 2q_4q_5 + 2q_4q_6 + 2q_4q_7 + 2q_4q_8 - 2q_4 + 2q_5q_6 + 2q_5q_7 + 2q_5q_9 - 2q_5 }$ 

${ +2q_6q_7 - 2q_6 - 2q_7 + 2q_8q_9 - 2q_8 - 2q_9 + 8}$ 






これをQuboマトリクスで表すと、以下のようになります。


<img src="https://user-images.githubusercontent.com/5043340/45666980-42f70f00-bb52-11e8-93a7-245e9d0f5609.png" width="400">


## 条件３： 移動距離が最小

ある都市から次の都市に移動するときの距離の和をコスト関数とすれば、コスト関数が最小のとき、移動距離が最短となるルートが求められます。例えば、q0（x11）とq5（x22）が１のとき、これは都市Aを１番目に訪れ、都市Bに２番目に訪れるということなので、都市Aから都市Bの移動に対応しています。そこで、Quboマトリクスのq0q5のコストは「2」となります。同様にすべての都市間の距離を考えると、下図のようになります。

<img src="https://user-images.githubusercontent.com/5043340/45667633-f3661280-bb54-11e8-9fbe-5dba63749b1d.png" width="400">


## 条件を足し合わせる

条件１・２、条件３を足し合わせたQuboマトリクスにします。

制約条件と距離のコスト関数は等しい割合で参入ができません。結合する際にはどちらかに係数をかけてその係数を調整する必要があります。今回は、距離のコスト関数側にB=0.25という係数をかけて足し合わせてみます。


## プログラムを書いて計算してみる


これをpythonプログラムで書き、シミュレータを実行して結果を得ます。


In [0]:
a.qubo=np.array([
  [-2,2,2,2,2,0,0,0,2,0,0,0,2,0,0,0],
  [0,-2,2,2,0,2,0,0,0,2,0,0,0,2,0,0],
  [0,0,-2,2,0,0,2,0,0,0,2,0,0,0,2,0],
  [0,0,0,-2,0,0,0,2,0,0,0,2,0,0,0,2],
  [0,0,0,0,-2,2,2,2,2,0,0,0,2,0,0,0],
  [0,0,0,0,0,-2,2,2,0,2,0,0,0,2,0,0],
  [0,0,0,0,0,0,-2,2,0,0,2,0,0,0,2,0],
  [0,0,0,0,0,0,0,-2,0,0,0,2,0,0,0,2],
  [0,0,0,0,0,0,0,0,-2,2,2,2,2,0,0,0],
  [0,0,0,0,0,0,0,0,0,-2,2,2,0,2,0,0],
  [0,0,0,0,0,0,0,0,0,0,-2,2,0,0,2,0],
  [0,0,0,0,0,0,0,0,0,0,0,-2,0,0,0,2],
  [0,0,0,0,0,0,0,0,0,0,0,0,-2,2,2,2],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,-2,2,2],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2,2],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2],
])+np.array([
  [0,0,0,0,0,2,0,2,0,1,0,1,0,3,0,3],
  [0,0,0,0,2,0,2,0,1,0,1,0,3,0,3,0],
  [0,0,0,0,0,2,0,2,0,1,0,1,0,3,0,3],
  [0,0,0,0,2,0,2,0,1,0,1,0,3,0,3,0],
  [0,0,0,0,0,0,0,0,0,4,0,4,0,2,0,2],
  [0,0,0,0,0,0,0,0,4,0,4,0,2,0,2,0],
  [0,0,0,0,0,0,0,0,0,4,0,4,0,2,0,2],
  [0,0,0,0,0,0,0,0,4,0,4,0,2,0,2,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2],
  [0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2],
  [0,0,0,0,0,0,0,0,0,0,0,0,2,0,2,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
  [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
])*0.25
answer = a.sa()

得られた結果を表示してみます。

In [4]:
print(answer)

[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0]


結果は

[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0]

となりました。４つごとに、都市A、B、C、Dを訪れる順番のところに1が設定されています。この結果によれば、都市A→C→D→B→Aの順に訪れることになります。計算すると移動距離は７ですので、最短のルートが計算されていることがわかります。